# Load modules, define model and detector

In [9]:
import torch
import torchvision.transforms as T
torch.set_grad_enabled(False);
from PIL import Image

# standard PyTorch mean-std input image normalization
transform = T.Compose([
  T.Resize(800),
  T.ToTensor(),
  T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

def detect(im, model, transform):
  # define help functions
  # for output bounding box post-processing
  def box_cxcywh_to_xyxy(x):
    x_c, y_c, w, h = x.unbind(1)
    b = [(x_c - 0.5 * w), (y_c - 0.5 * h),
         (x_c + 0.5 * w), (y_c + 0.5 * h)]
    return torch.stack(b, dim=1)

  def rescale_bboxes(out_bbox, size):
    img_w, img_h = size
    b = box_cxcywh_to_xyxy(out_bbox)
    b = b * torch.tensor([img_w, img_h, img_w, img_h], dtype=torch.float32)
    return b

  # mean-std normalize the input image (batch-size: 1)
  img = transform(im).unsqueeze(0)

  # propagate through the model
  outputs = model(img)

  # keep only predictions with 0.7+ confidence
  probas = outputs['pred_logits'].softmax(-1)[0, :, :-1]
  keep = probas.max(-1).values > 0.7

  # convert boxes from [0; 1] to image scales
  bboxes_scaled = rescale_bboxes(outputs['pred_boxes'][0, keep], im.size)
  return probas[keep], bboxes_scaled

# import model from pytorch model hub
model = torch.hub.load('facebookresearch/detr', 'detr_resnet50', pretrained=True)
print(f'CUDA: {torch.cuda.is_available()} -> {torch.cuda.get_device_name(0)}')

Using cache found in /home/max/.cache/torch/hub/facebookresearch_detr_master


CUDA: True -> GeForce GTX 1050 Ti


In [2]:
import numpy as np
import pandas as pd
import json
import requests
import time
import os
from SPARQLWrapper import SPARQLWrapper, JSON

def get_sparql_dataframe(service, query):
    """
    Helper function to convert SPARQL results into a Pandas data frame.
    """
    sparql = SPARQLWrapper(service)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    result = sparql.query()

    processed_results = json.load(result.response)
    cols = processed_results['head']['vars']

    out = []
    for row in processed_results['results']['bindings']:
        item = []
        for c in cols:
            item.append(row.get(c, {}).get('value'))
        out.append(item)

    return pd.DataFrame(out, columns=cols)

q = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX wd: <http://www.wikidata.org/entity/>
PREFIX : <http://h-da.de/fbi/artontology/>

SELECT ?artwork ?url ?name ?abstract (group_concat(?motif;separator=';') as ?motifs)
WHERE {
  ?artwork rdf:type :artwork;
     rdfs:label ?name;
     :image ?url;
     :abstract ?abstract;
     :motif/rdfs:label ?motif.
}
group by ?artwork ?url ?name ?abstract
"""

ds = "http://neuds.de:3030/artontology"
df = get_sparql_dataframe(ds, q)
df['id'] = df['artwork'].map(lambda x: x.split('/')[-1])
df.head()

,artwork,url,name,abstract,motifs,id
0,http://www.wikidata.org/entity/Q29019486,https://upload.wikimedia.org/wikipedia/commons...,Saint Proculus of Pozzuoli and his mother Sant...,Saints Proculus and Nicea is a 1636-1637 paint...,Nicea;Saint Procolo;Saint Procolo and Nicea,Q29019486
1,http://www.wikidata.org/entity/Q7527543,https://upload.wikimedia.org/wikipedia/commons...,Statue of John A. Macdonald,The Sir John A. Macdonald statue is a bronze s...,John A. Macdonald,Q7527543
2,http://www.wikidata.org/entity/Q3944494,https://upload.wikimedia.org/wikipedia/commons...,The Holy Family with Saint Catherine of Alexan...,Holy Family with Saint Catherine of Alexandria...,Virgin Mary;Catherine of Alexandria;boy;Child ...,Q3944494
3,http://www.wikidata.org/entity/Q368788,https://upload.wikimedia.org/wikipedia/commons...,Pietà,Pietà is a painting by the Italian Renaissance...,Virgin Mary;Jesus Christ;man;woman,Q368788
4,http://www.wikidata.org/entity/Q3630743,https://upload.wikimedia.org/wikipedia/commons...,Self-Portrait at the Age of 63,Self-Portrait at the Age of 63 is a self-portr...,Rembrandt;man,Q3630743


# Inference

run inference

In [3]:
from gensim.matutils import cossim as csim

In [4]:
def get_scores(aid, df):
  dir = 'res'
  ext = df[df['id'] == aid]['url'].values[0].split('.')[-1].lower()
  file = f'{dir}/{aid}.{ext}'
  try:
    im = Image.open(file)
    scores, _ = detect(im, model, transform)
    if scores.shape[0] > 0:
      scores = scores.detach().numpy()[0]
      bow = {}
      for i, score in enumerate(scores):
        if score > 0.6:
          if i in bow:
            bow[i] += 1
          else:
            bow[i] = 1
    else:
      bow = {0:0}
  except:
    bow = {}
  return(bow)

def get_sims(id0, id1):
  scores0 = get_scores(id0, df)
  scores1 = get_scores(id1, df)
  sims = csim(scores0, scores1)
  return(np.around(sims, 3))

fill bocv

In [ ]:
bocv = {}
path_bocv = 'art_bocv.json'
with open(path_bocv) as f:
  bocv = json.load(f)
counter = 0
for key in bocv:
  if len(bocv[key]) == 0:
    bow = get_scores(key, df)
    bocv[key] = bow
    counter += 1
    print(f'Done: {key}')
  if counter == 5:
    with open(path_bocv, 'w') as f:
      json.dump(bocv, f)
    counter = 0
    print('dumped bocv')
with open(path_bocv, 'w') as f:
  json.dump(bocv, f)
print('dumped bocv')

Done: Q2234397
Done: Q17609277
Done: Q738038
Done: Q7232301
Done: Q17182872
dumped bocv
Done: Q6583495
Done: Q3929322
Done: Q52432
Done: Q1990128
Done: Q19697889
dumped bocv
Done: Q19912578
Done: Q19326698
Done: Q2415259
Done: Q3842580
Done: Q18600184
dumped bocv
Done: Q39073118
Done: Q1760564
Done: Q19912015
Done: Q18736756
Done: Q649312
dumped bocv
Done: Q742801
Done: Q254923
Done: Q2885144
Done: Q14940739
Done: Q11129389
dumped bocv
Done: Q18511742
Done: Q2488682
Done: Q3201475
Done: Q926536
Done: Q3947210
dumped bocv
Done: Q15934578
Done: Q2993019
Done: Q1133457
Done: Q1195238
Done: Q3208973
dumped bocv
Done: Q19905377
Done: Q24213
Done: Q1254326
Done: Q17347232
Done: Q5189814
dumped bocv
Done: Q2628761
Done: Q1551926
Done: Q3699475
Done: Q6728373
Done: Q17435192
dumped bocv
Done: Q3937473
Done: Q2268013
Done: Q18573258
Done: Q29385758
Done: Q22329818
dumped bocv
Done: Q15714659
Done: Q3612419
Done: Q2980775
Done: Q24930566
Done: Q3204269
dumped bocv
Done: Q13726073
Done: Q1217213


Done: Q7972064
Done: Q2733729
Done: Q27980928
Done: Q669872
Done: Q3204226
dumped bocv
Done: Q2918420
Done: Q28541710
Done: Q18226124
Done: Q4373352
Done: Q3605543
dumped bocv
Done: Q17341499
Done: Q1516449
Done: Q523974
Done: Q3797773
Done: Q15974342
dumped bocv
Done: Q21723151
Done: Q17030253
Done: Q9019507
Done: Q18748777
Done: Q7934004
dumped bocv
Done: Q16582179
Done: Q18162015
Done: Q3208369
Done: Q19924428
Done: Q17175813
dumped bocv
Done: Q664038
Done: Q3733361
Done: Q5531755
Done: Q3212521
Done: Q2200610
dumped bocv
Done: Q2203011
Done: Q18159614
Done: Q27919854
Done: Q1448701
Done: Q1272037
dumped bocv
Done: Q3079004
Done: Q979440
Done: Q2629134
Done: Q2303840
Done: Q6106757
dumped bocv
Done: Q3907490
Done: Q5561848
Done: Q18344787
Done: Q10875359
Done: Q3944472
dumped bocv
Done: Q688870
Done: Q67847999
Done: Q3099894
Done: Q20166980
Done: Q6240450
dumped bocv
Done: Q602813
Done: Q572697
Done: Q6470337
Done: Q21286417
Done: Q2513438
dumped bocv
Done: Q920030
Done: Q1167932
Do

In [51]:
id0 = 'Q12418'
id1 = 'Q39226'
sims = get_sims(id0, id1)
print(sims)

1.0


In [7]:
torch.cuda.get_device_name(0)

'GeForce GTX 1050 Ti'